In [1]:
import os
import sys
import matplotlib.pyplot as plt
from pyproj import CRS
from pyresample import geometry
import numpy as np
import xarray as xr
from PIL import Image
import copy

from skimage import data, img_as_float
from skimage import exposure

import satpy
import satpy.resample

from satpy.composites import SingleBandCompositor, GenericCompositor
#from trollimage.image import Image
from satpy.writers import to_image
from trollimage.colormap import greys, gnbu, bugn, greens, orrd, rdylgn, pubugn, ylgn, brbg
#from trollimage.image import Image
from pycoast import ContourWriterAGG

from scipy.signal import find_peaks

import glob
from satpy import Scene

from pyresample.geometry import SwathDefinition

sys.path.insert(0, '/home/cameron/Projects/hypso-package')

# Import Satellite Object
from hypso import Hypso1

In [2]:
if True:
    dir_path = '/home/cameron/Dokumenter/Data/frohavet'
    nc_file = os.path.join(dir_path, 'frohavet_2024-05-06_1017Z-l1a.nc')
    points_file = os.path.join(dir_path, 'frohavet_2024-05-06_1017Z-bin3.points')
    cloud_mask_quantile = 0.6
    sentinel_3_dir = "/home/cameron/Dokumenter/Frohavet_Sentinel_3/S3B_OL_2_WFR____20240506T102040_20240506T102340_20240506T122602_0179_092_336_1800_MAR_O_NR_003.SEN3"

if False:
    dir_path = '/home/cameron/Dokumenter/Data/erie'
    nc_file = os.path.join(dir_path, 'erie_2022-07-20_1539Z-l1a.nc')
    points_file = os.path.join(dir_path, 'erie_2022-07-20_1539Z-bin3.points')

if False:
    dir_path = '/home/cameron/Dokumenter/Data/frohavet'
    nc_file = os.path.join(dir_path, 'frohavet_2024-04-26_1049Z-l1a.nc')
    points_file = os.path.join(dir_path, 'frohavet_2024-04-26_1049Z-bin3.points')

if False:
    dir_path = "/home/cameron/Dokumenter/Data/roervik/"
    nc_file = os.path.join(dir_path, 'roervik_2024-05-05_1039Z-l1a.nc')
    points_file = os.path.join(dir_path, 'roervik_2024-05-05_1039Z-bin3.points')
    cloud_mask_quantile = 0.85
    sentinel_3_dir = "/home/cameron/Dokumenter/Frohavet_Sentinel_3/S3B_OL_2_WRR____20240505T103843_20240505T112305_20240506T142550_2662_092_322______MAR_O_NT_003.SEN3"

if False:
    dir_path = "/home/cameron/Dokumenter/Data/frohavet/"
    nc_file = os.path.join(dir_path, 'frohavet_2024-04-26_1049Z-l1a.nc')
    points_file = os.path.join(dir_path, 'frohavet_2024-04-26_1049Z-bin3.points')
    cloud_mask_quantile = 0.5
    sentinel_3_dir = "/home/cameron/Dokumenter/Frohavet_Sentinel_3/S3B_OL_2_WRR____20240426T093209_20240426T101628_20240427T120354_2659_092_193______MAR_O_NT_003.SEN3"


In [ ]:
satobj = Hypso1(hypso_path=nc_file, points_path=points_file, verbose=True)
satobj.write_l1a_nc_file()
satobj.generate_l1b_cube(compute_destriping_matrix=True)
#l1b_scene = satobj.get_l1b_satpy_scene()
#satobj.generate_l1b_cube(compute_destriping_matrix=False, overwrite=True)
#l1b_scene_2 = satobj.get_l1b_satpy_scene()
#satobj.generate_geometry()
#satobj.write_l1b_nc_file()
#satobj.generate_l2a_cube(product_name='acolite')

In [ ]:
satobj.generate_chlorophyll_estimates('band_ratio')

In [ ]:
#l1a_scene = satobj.get_l1a_satpy_scene()
#l1b_scene = satobj.get_l1b_satpy_scene()
#l2a_scene = satobj.get_l2a_satpy_scene()
#chl_scene = satobj.get_chlorophyll_satpy_scene()
products_scene = satobj.get_products_satpy_scene()
#satobj.generate_geometry()

In [ ]:
chl_scene['chl_band_ratio']

In [ ]:
satobj.products

In [ ]:
satobj.write_l1b_nc_file()

In [ ]:
original_path = satobj.l1b_nc_file
file_name = original_path.name
modified_file_name = file_name.replace('-l1b', '-l1b-modified')
modified_path = original_path.with_name(modified_file_name)

In [ ]:
print(modified_path)

In [ ]:
# RGB Hypso image
compositor = GenericCompositor("rgb")

#R:630nm, G:550nm, B:480nm
# Red, Green, Blue, Alpha
red_wl = 630
green_wl = 550
blue_wl = 480

red_idx = satobj.get_closest_wavelength_index(red_wl)
green_idx = satobj.get_closest_wavelength_index(green_wl)
blue_idx = satobj.get_closest_wavelength_index(blue_wl)

red_band = 'band_' + str(red_idx)
green_band = 'band_' + str(green_idx)
blue_band = 'band_' + str(blue_idx)

composite = compositor([l1b_scene_1[red_band][:,::3], 
                        l1b_scene_1[green_band][:,::3], 
                        l1b_scene_1[blue_band][:,::3], 
                        l1b_scene_1['band_5'][:,::3]]) 

rgb_img = to_image(composite) 
rgb_img.stretch_linear()
rgb_img.gamma(2)

rgb_xr_image = rgb_img

rgb_img = rgb_img.pil_image()

rgb_img.save('./' + satobj.capture_name + '_rgb_only_computed.png')

In [ ]:
rgb_img

In [ ]:
rgb_img

In [ ]:
# RGB Hypso image
compositor = GenericCompositor("rgb")

#R:630nm, G:550nm, B:480nm
# Red, Green, Blue, Alpha
red_wl = 630
green_wl = 550
blue_wl = 480

red_idx = satobj.get_closest_wavelength_index(red_wl)
green_idx = satobj.get_closest_wavelength_index(green_wl)
blue_idx = satobj.get_closest_wavelength_index(blue_wl)

red_band = 'band_' + str(red_idx)
green_band = 'band_' + str(green_idx)
blue_band = 'band_' + str(blue_idx)

composite = compositor([l1b_scene_2[red_band][:,::3], 
                        l1b_scene_2[green_band][:,::3], 
                        l1b_scene_2[blue_band][:,::3], 
                        l1b_scene_2['band_5'][:,::3]]) 

rgb_img = to_image(composite) 
rgb_img.stretch_linear()
rgb_img.gamma(2)

rgb_xr_image = rgb_img

rgb_img = rgb_img.pil_image()

rgb_img.save('./' + satobj.capture_name + '_rgb_only_csv.png')

In [ ]:
rgb_img

In [ ]:
rgb_img

In [ ]:
from pyproj import Transformer
from pyproj import Proj

area_id = satobj.capture_region
proj_id = 'roi'
description = 'roi'

bbox = satobj.get_bbox()
lon_0 = (bbox[2] - bbox[0])/2 + bbox[0]
lat_ts = (bbox[3] - bbox[1])/2 + bbox[1]

p = Proj(proj="stere", ellps="bessel", lat_0=90.0, lon_0=lon_0, lat_ts=lat_ts)
#Transformer.from_proj
#p(12.06, 65.26)
#p(-100000, -2950000, inverse=True)

projection = {"proj": "stere", 
              "ellps": "bessel", 
              "lat_0": 90.0, 
              "lon_0": lon_0, 
              "lat_ts": lat_ts, 
              "units": "m"}

#projection = CRS.from_proj4("+proj=latlon")
#projection = p.to_proj4()

swath_def = SwathDefinition(lons=satobj.longitudes, lats=satobj.latitudes)
projection = swath_def.compute_bb_proj_params(proj_dict=projection)

lower_left_x, lower_left_y = p(bbox[0], bbox[1])
upper_right_x, upper_right_y = p(bbox[2], bbox[3])
area_extent = (lower_left_x, lower_left_y, upper_right_x, upper_right_y)

res = 300 #meters

width = (area_extent[2] - area_extent[0]) / res
height = (area_extent[3] - area_extent[1]) / res

area_def = geometry.AreaDefinition(area_id, proj_id, description, projection,  width, height, area_extent)

#from pyresample import load_area
#area_def = load_area("./frohavet.yaml")

In [ ]:
resampled_chl_scene = None
resampled_l1a_scene = None
size = None

resampled_l1a_scene = l1b_scene.resample(area_def, resampler='bilinear', fill_value=np.NaN)
#resampled_l1b_scene = l1b_scene.resample(area_def, resampler='bilinear', fill_value=np.NaN)
#resampled_chl_scene = chl_scene.resample(area_def, resampler='bilinear', fill_value=np.NaN)

In [ ]:
size = (area_def.shape[1], area_def.shape[0])

# Base image
base_img = Image.new(mode="RGBA", size=size, color=(173, 216, 230))
cw = ContourWriterAGG('/home/cameron/Dokumenter/Pycoast')
cw.add_coastlines(base_img, area_def, resolution='f', outline='black', fill='gray')

# Overlay image
overlay_img = Image.new(mode="RGBA", size=size)
cw = ContourWriterAGG('/home/cameron/Dokumenter/Pycoast')
cw.add_coastlines(overlay_img, area_def, resolution='f', outline='black', level=4)
cw.add_rivers(overlay_img, area_def, level=10, outline='blue', resolution='f')
#cw.add_borders(overlay_img, area_def, outline=(255, 0, 0))


boundary_lats, boundary_lons = swath_def.get_boundary_lonlats()

side1 = tuple((lat, lon) for lat, lon in zip(boundary_lats.side1, boundary_lons.side1))
side2 = tuple((lat, lon) for lat, lon in zip(boundary_lats.side2, boundary_lons.side2))
side3 = tuple((lat, lon) for lat, lon in zip(boundary_lats.side3, boundary_lons.side3))
side4 = tuple((lat, lon) for lat, lon in zip(boundary_lats.side4, boundary_lons.side4))

polygon_coords = side1 + side2 + side3 + side4

#from shapely import Polygon
#polygon = Polygon(polygon_coords)

# Overlay image
boundary_img = Image.new(mode="RGBA", size=size)
cw = ContourWriterAGG('/home/cameron/Dokumenter/Pycoast')
cw.add_polygon(boundary_img, area_def, polygon_coords, outline='red', width=3)

In [ ]:
# RGB Hypso image
compositor = GenericCompositor("rgb")

#R:630nm, G:550nm, B:480nm
# Red, Green, Blue, Alpha
red_wl = 630
green_wl = 550
blue_wl = 480

red_idx = satobj.get_closest_wavelength_index(red_wl)
green_idx = satobj.get_closest_wavelength_index(green_wl)
blue_idx = satobj.get_closest_wavelength_index(blue_wl)

red_band = 'band_' + str(red_idx)
green_band = 'band_' + str(green_idx)
blue_band = 'band_' + str(blue_idx)

composite = compositor([resampled_l1a_scene[red_band], 
                        resampled_l1a_scene[green_band], 
                        resampled_l1a_scene[blue_band], 
                        resampled_l1a_scene['band_5']]) 

rgb_img = to_image(composite) 
rgb_img.stretch_linear()
rgb_img.gamma(2)

rgb_xr_image = rgb_img

rgb_img = rgb_img.pil_image()

rgb_img.save('./' + satobj.capture_name + '_rgb_only.png')

In [ ]:
#plotting_cm = ylgn
plotting_cm = bugn

In [ ]:
chl_product = 'chl_band_ratio'
#chl_product = 'chl_6sv1_aqua'
#chl_min = 0
#chl_max = np.nanmax(resampled_chl_scene[chl_product])

#chl_product = 'chl_band_ratio'
#chl_min = 1.9
#chl_max = 2.2

#chl_product = 'chl_6sv1_aqua'
#chl_min = 0
#chl_max = 2

resampled_chl_scene['chl_plotted'] = copy.deepcopy(resampled_chl_scene[chl_product])

values = None
del values
values = None

values = copy.deepcopy(resampled_chl_scene[chl_product].to_numpy())
#values[values<chl_min] = np.nan
#values[values>chl_max] = np.nan
#values = np.log10(values)
#values = 10**values

# Contrast stretching
if True:
    p2, p98 = np.nanpercentile(values, (2, 98))
    values = exposure.rescale_intensity(values, in_range=(p2, p98))

# Equalization
if False:
    mask = np.isnan(values)
    values = exposure.equalize_hist(values, mask=~mask)

# Adaptive Equalization
if False:

    # Scale between -1 and 1
    values = 2 * (values - np.nanmin(values)) / (np.nanmax(values) - np.nanmin(values)) - 1

    # Create a mask for NaN values
    nan_mask = np.isnan(values)

    # Fill NaN values with a neutral value (e.g., the mean of the non-NaN elements)
    values_no_nan = np.copy(values)
    mean_value = np.nanmean(values_no_nan)
    values_no_nan[nan_mask] = mean_value

    # Apply adaptive histogram equalization
    values_eq = exposure.equalize_adapthist(values_no_nan, clip_limit=0.01)

    # Reapply the NaN mask to the equalized image
    values_eq[nan_mask] = np.nan

    values = values_eq

if False:
    from skimage.filters.rank import mean
    from skimage.morphology import disk

    # Create a mask for NaN values
    nan_mask = np.isnan(values)

    # Fill NaN values with a neutral value (e.g., the mean of the non-NaN elements)
    values_no_nan = np.copy(values)
    mean_value = np.nanmean(values_no_nan)
    values_no_nan[nan_mask] = mean_value
    values_smoothed = mean(values_no_nan, disk(7))
    values_smoothed = values_smoothed.astype('float')
    values_smoothed[nan_mask] = np.nan
    values = values_smoothed


values = (values - np.nanmin(values)) / (np.nanmax(values) - np.nanmin(values)) 
#values = 2 * (values - np.nanmin(values)) / (np.nanmax(values) - np.nanmin(values)) - 1

resampled_chl_scene['chl_plotted'].values = values

#chl_min = 0.2
#chl_max = 1.2

#chl_min = 0.1
#chl_max = 1.1

chl_min = 0
chl_max = 1.1

chl_ds = resampled_chl_scene['chl_plotted']


compositor = SingleBandCompositor('chl')
composite = compositor([chl_ds])
chl_img = to_image(composite) 
chl_cm = plotting_cm.set_range(chl_min,chl_max,inplace=False)
chl_cm.reverse()
chl_img.colorize(chl_cm)
chl_img = chl_img.pil_image()


chl_img_2 = to_image(composite) 
chl_cm = plotting_cm.set_range(chl_min,chl_max,inplace=False)
chl_cm.reverse()
chl_img_2.colorize(chl_cm)
chl_img_2 = chl_img_2.pil_image()

combined_img = Image.alpha_composite(base_img, rgb_img)
combined_img = Image.alpha_composite(base_img, chl_img)
combined_img = Image.alpha_composite(combined_img, boundary_img)
combined_img = Image.alpha_composite(combined_img, overlay_img)
combined_img.save('./' + satobj.capture_name + '_hypso_chl.png')


from pydecorate import DecoratorAGG

combined_img = Image.alpha_composite(base_img, rgb_img)
combined_img = Image.alpha_composite(combined_img, boundary_img)
combined_img = Image.alpha_composite(combined_img, overlay_img)



if False:
    chl_cm = plotting_cm.set_range(chl_min,chl_max,inplace=False)
    dc = DecoratorAGG(combined_img)
    dc.write_vertically()
    dc.align_right()
    dc.align_bottom()

    #dc.add_text("HYPSO-1 HSI\n" + str(satobj.capture_name), height=180)
    #dc.add_logo("../assets/HYPSO_logo.png")

    dc.add_scale(chl_cm, extend=False, max_val=1, height = 512, margins=[10,10])


combined_img.save('./' + satobj.capture_name + '_hypso_rgb.png')

combined_img = Image.alpha_composite(base_img, rgb_img)
combined_img = Image.alpha_composite(combined_img, chl_img_2)
combined_img = Image.alpha_composite(combined_img, boundary_img)
combined_img = Image.alpha_composite(combined_img, overlay_img)
combined_img.save('./' + satobj.capture_name + '_hypso_combined.png')

In [ ]:

filenames = []
filenames = filenames + glob.glob(sentinel_3_dir + '/geo_coordinates.nc')
filenames = filenames + glob.glob(sentinel_3_dir + '/chl_nn.nc')
filenames = filenames + glob.glob(sentinel_3_dir + '/tsm_nn.nc')

s3_scene = Scene(filenames=filenames, reader='olci_l2')

s3_scene.load(['chl_nn', 'tsm_nn', 'latitude', 'longitude'])

s3_scene['chl_nn'].data = 10**s3_scene['chl_nn'].data

resampled_s3_scene = s3_scene.resample(area_def, resampler='bilinear', fill_value=np.NaN)

In [ ]:
s3_min = 0
s3_max = 10
#chl_max = np.nanmax(chl_scene[chl_product].to_numpy())

compositor = SingleBandCompositor('s3_chl')
composite = compositor([resampled_s3_scene['chl_nn']])
s3_img = to_image(composite) 
s3_cm = plotting_cm.set_range(min_val=s3_min, max_val=s3_max, inplace=True)
#s3_cm.reverse()
s3_img.colorize(s3_cm)
s3_img = s3_img.pil_image()

combined_s3_img = Image.alpha_composite(base_img, s3_img)
combined_s3_img = Image.alpha_composite(combined_s3_img, boundary_img)
combined_s3_img = Image.alpha_composite(combined_s3_img, overlay_img)

combined_s3_img.save('./' + satobj.capture_name + '_sentinel3_chl.png')

In [ ]:
plt.imshow(satobj.l1a_cube[:,:,20])

In [ ]:
plt.imshow(satobj.l1b_cube[:,:,20])

In [ ]:
c = np.zeros((satobj.l1b_cube.shape[0],satobj.l1b_cube.shape[1]))

c[:,:] = satobj.destriping_coeffs[:,20]


In [ ]:
plt.imshow(c)

In [ ]:
datacube = satobj.l1a_cube.to_numpy()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

height, width, bands = datacube.shape
reshaped_data = datacube.reshape(-1, bands)

# Apply PCA
pca = PCA(n_components=5)
pca_data = pca.fit_transform(reshaped_data)

# Reconstruct the data without stripe-related components
#reconstructed_data = pca.inverse_transform(pca_data)
#destriped_cube = reconstructed_data.reshape(height, width, bands)


In [ ]:
pca_comps = pca_data.reshape(height, width, 5)

In [ ]:
plt.imshow(pca_comps[:,:,2])

In [ ]:
pca_comps[:,:,2].attrs

In [ ]:
class NumpyArrayDict(dict):
    def __setitem__(self, key, value):
        """Override the method for setting a dictionary item."""
        if not isinstance(value, np.ndarray):
            raise TypeError(f"Value must be of type np.ndarray, but got {type(value).__name__}")
        super().__setitem__(key, value)

    def update(self, *args, **kwargs):
        """Override the update method to check all added values."""
        if args:
            other = dict(args[0]) if len(args) == 1 else dict(args)
            for key, value in other.items():
                self[key] = value
        for key, value in kwargs.items():
            self[key] = value

In [ ]:
n = NumpyArrayDict()

In [ ]:
n['hello'] = np.zeros(2)

In [ ]:
n['hello']

In [ ]:
n.keys()

In [ ]:
type(n)

In [ ]:
z = [1,2,3]

In [ ]:
z[:2]

In [ ]:
from hypso.DataArrayDict import DataArrayDict

In [ ]:
da = DataArrayDict(attributes={}, dims_shape=satobj.spatial_dimensions)

In [ ]:
from hypso.reading import load_l1a_nc_cube

In [ ]:
value = load_l1a_nc_cube(satobj.hypso_path)

In [ ]:
da['default'] = value

In [ ]:
da